## Installing libraries

In [ ]:
!pip install cohere
!pip install langchain
!pip install pypdf
!pip install tiktoken
!pip install chromadb
!pip install bs4

## read url source file

In [ ]:
def read_urls(file_path):
    '''Function to read URLs from a file'''
    with open(file_path, 'r') as file:
        urls = [url.strip() for url in file.readlines()]
    return urls

urls = read_urls("./source_url.txt")

## load the url source

In [ ]:
from langchain.document_loaders import WebBaseLoader

# Specify the URL of the website you want to load
all_documents = []

# Load the website content for each URL and add it to the list
for url in urls:
    loader = WebBaseLoader(url)
    documents = loader.load()
    all_documents.extend(documents)

all_documents

## load document from local file

In [4]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./source.txt')
all_documents = loader.load()

all_documents

[Document(page_content='Trades:\n744\nProfit Trades:\n490 (65.86%)\nLoss Trades:\n254 (34.14%)\nBest trade:\n57.45 USD\nWorst trade:\n-45.59 USD\nGross Profit:\n3 246.73 USD (344 065 pips)\nGross Loss:\n-1 705.90 USD (159 746 pips)\nMaximum consecutive wins:\n43 (101.14 USD)\nMaximal consecutive profit:\n124.46 USD (4)\nSharpe Ratio:\n0.19\nTrading activity:\n100.00%\nMax deposit load:\n8.24%\nLatest trade:\n3 days ago\nTrades per week:\n15\nAvg holding time:\n7 days \nRecovery Factor:\n12.02\nLong Trades:\n354 (47.58%)\nShort Trades:\n390 (52.42%)\nProfit Factor:\n1.90\nExpected Payoff:\n2.07 USD\nAverage Profit:\n6.63 USD\nAverage Loss:\n-6.72 USD\nMaximum consecutive losses:\n8 (-34.22 USD)\nMaximal consecutive loss:\n-75.82 USD (6)\nMonthly growth:\n5.89%\nAnnual Forecast:\n72.88%\nAlgo trading:\n73%', metadata={'source': './source.txt'})]

## split the website content into smaller chunks

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os
import cohere

key = "urAN0hpVmnaPAW66RPqZQTKOSs46zd4EI93e1NNg"
co = cohere.Client(key)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_documents)

os.environ["COHERE_API_KEY"] = key

## store vector

In [6]:
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(cohere_api_key=key)
db = Chroma.from_documents(docs, embeddings)

## define prompt template

In [7]:
from langchain.prompts import PromptTemplate
prompt_template = """
you are a personal financial advisor here to help the client. 

Given the following financial context:

{context}

Question: {question}
As your financial advisor, I would recommend the following:
"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

## retrieve dara and output

In [8]:
from langchain.chains import RetrievalQA
from langchain.llms import Cohere

retriever = db.as_retriever()  # Assuming 'db' is your vector store

retrievalQA = RetrievalQA.from_llm(
    llm=Cohere(),
    retriever=retriever,
    prompt=prompt
)

/Users/apple/Desktop/rag-chatbot/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


## query about the urls content

In [9]:
# query = "What is an activation?"
# retrievalQA.invoke(query)

## query about the source file content

In [10]:
query = "can you summarize my trading performance?"
retrievalQA.invoke(query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'query': 'can you summarize my trading performance?',
 'result': " Given the provided trading context and performance summary, I can provide you with some insights and recommendations:\n\n1. Trading Performance - You have traded 744 times with 490 profitable trades, which is a strong 65.86% win rate. This is an admirable performance, but it's important to consider the risk-reward ratio to make sustainable long-term gains. \n\n2. Risk Management - While your best trade yielded 57.45 USD, the worst trade was -45.59 USD, suggesting a significant loss incurred at times. It is important to manage your risks effectively, and your risk per trade should be in line with your risk tolerance and trading goals. \n\n3. Holding Time - With an average holding time of 7 days, your trades are positioned in the short-term trading realm, which often entails higher exposure to market volatility and potential quick changes in market conditions. Adequate monitoring of trades and a defined exit strategy are

## invalid query

In [ ]:
invalid_query = "where can i buy icecream?"
retrievalQA.invoke(invalid_query)